In [53]:
import pandas as pd
import numpy as np
import seaborn as sb
import datetime
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from get_decision_variable_map import get_decision_variable_map
from get_case_outputs_all_models import get_case_outputs_all_models
from get_unique_resources_data import get_unique_resources_data
from get_printable_resource_names import get_printable_resource_names

In [54]:
import os

current_dir = os.getcwd()
print(current_dir)

c:\Users\ks885\Documents\aa_research\Modeling\spcm_genx_experiment\figures


In [55]:
plots_path = os.path.join(current_dir, 'plots') + "/"
pdf_path = os.path.join(current_dir, 'pdf_tables') + "/"
latex_path = os.path.join(current_dir, 'latex') + "/"
csv_path = os.path.join(current_dir, 'csv') + "/"
jpg_path = os.path.join(current_dir, 'jpg') + "/"

In [56]:
revenue_band_path = os.path.join(jpg_path, 'revenue_band') + "/"
if not os.path.exists(revenue_band_path):
    os.makedirs(revenue_band_path)

In [57]:
# modeling scaling ModelScalingFactor
ModelScalingFactor = 1000

cem_path = os.path.join(os.path.dirname(current_dir), 'GenX.jl', 'research_systems')
policies_path = os.path.join(os.path.dirname(current_dir), 'SPCM', 'research_systems')

date = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [58]:
case_names = [    
              "Thermal_Base",
              "2_Hr_BESS", 
              "2_Hr_BESS_Fuelx2",
              "4_Hr_BESS",
              "4_Hr_BESS_Fuelx2",
              "4_Hr_BESS_Fuelx3",
              "4_Hr_BESS_Fuelx4",
              "6_Hr_BESS",
              "6_Hr_BESS_Fuelx2",
              "8_Hr_BESS",
              "8_Hr_BESS_Fuelx2",
              "10_Hr_BESS",
              "10_Hr_BESS_Fuelx2",
              ]

policy_types = [
                'pf',
                'dlac-p',
                'dlac-i',
                'slac',
]

# Create a DataFrame with decision_variable_names as the index
decision_variable_map = get_decision_variable_map()

In [59]:
unique_resources, cases_resources_capacities = get_unique_resources_data(case_names, policies_path)


adding resource: NG 2-on-1 Combined Cycle (F-Frame) from case: Thermal_Base
adding resource: NG Combustion Turbine (F-Frame) from case: Thermal_Base
adding resource: Land-Based Wind - Class 1 - Technology 1 from case: Thermal_Base
adding resource: Utility PV - Class 1 from case: Thermal_Base
adding resource: Utility-Scale Battery Storage - 2Hr from case: 2_Hr_BESS
adding resource: Utility-Scale Battery Storage - 4Hr from case: 4_Hr_BESS
adding resource: Utility-Scale Battery Storage - 6Hr from case: 6_Hr_BESS
adding resource: Utility-Scale Battery Storage - 8Hr from case: 8_Hr_BESS
adding resource: Utility-Scale Battery Storage - 10Hr from case: 10_Hr_BESS


In [60]:
print_unique_resources = get_printable_resource_names(unique_resources)

In [61]:
print_unique_resources

['Wind', 'Solar', 'NG CC', 'BESS', 'NG CT']

In [62]:
# intialize dictionary for saving results for each case
skew_dict = {case: {} for case in case_names}
std_dict = {case: {} for case in case_names}

In [63]:
for case_name in case_names:

    print('Case Name: ' + case_name + '\n')


    cem_prices, policies_prices_dict, prices_cols = get_case_outputs_all_models(cem_path, 
        policies_path, case_name, 'energy prices', policy_types)
    

    pf_prices = policies_prices_dict['pf']
    dlac_prices = policies_prices_dict['dlac-p']
    dlac_imperfect_prices = policies_prices_dict['dlac-i']
    slac_prices = policies_prices_dict['slac']
    
    # get zone of prices
    zone_name = prices_cols[0]

    # calculate the skew of prices for each model
    cem_skew = cem_prices[zone_name].skew()
    pf_skew = pf_prices[zone_name].skew()
    dlac_skew = dlac_prices[zone_name].skew()
    dlac_imperfect_skew = dlac_imperfect_prices[zone_name].skew()
    slac_skew = slac_prices[zone_name].skew()


    skew_df = {'CEM': cem_skew, 'PF': pf_skew, 'DLAC-p': dlac_skew, 'DLAC-i': dlac_imperfect_skew, 'SLAC': slac_skew}

    # calculate the standard deviation of prices for each model
    cem_std = cem_prices[zone_name].std()
    pf_std = pf_prices[zone_name].std()
    dlac_std = dlac_prices[zone_name].std()
    dlac_imperfect_std = dlac_imperfect_prices[zone_name].std()
    slac_std = slac_prices[zone_name].std()

    std_df = {'CEM': cem_std, 'PF': pf_std, 'DLAC-p': dlac_std, 'DLAC-i': dlac_imperfect_std, 'SLAC': slac_std}

    # save results to dictionary
    skew_dict[case_name] = skew_df
    std_dict[case_name] = std_df
# end of loop    

Case Name: Thermal_Base

Case Name: 2_Hr_BESS

Case Name: 2_Hr_BESS_Fuelx2

Case Name: 4_Hr_BESS

Case Name: 4_Hr_BESS_Fuelx2

Case Name: 4_Hr_BESS_Fuelx3

Case Name: 4_Hr_BESS_Fuelx4

Case Name: 6_Hr_BESS

Case Name: 6_Hr_BESS_Fuelx2

Case Name: 8_Hr_BESS

Case Name: 8_Hr_BESS_Fuelx2

Case Name: 10_Hr_BESS

Case Name: 10_Hr_BESS_Fuelx2



In [64]:
def percent_change_df(df):
    norm = df.copy()
    norm['CEM'] = df['CEM'].astype(float) / df['PF'].astype(float)
    norm['PF'] = df['PF'].astype(float) / df['PF'].astype(float)
    norm['DLAC-p'] = df['DLAC-p'].astype(float) / df['PF'].astype(float)
    norm['DLAC-i'] = df['DLAC-i'].astype(float) / df['PF'].astype(float)
    norm['SLAC'] = df['SLAC'].astype(float) / df['PF'].astype(float)

    change_df = norm.copy()
    change_df['CEM'] = norm['CEM'] - 1
    change_df['PF'] = norm['PF'] - 1
    change_df['DLAC-p'] = norm['DLAC-p'] - 1
    change_df['DLAC-i'] = norm['DLAC-i'] - 1
    change_df['SLAC'] = norm['SLAC'] - 1

    percent_change_df = change_df.copy()
    percent_change_df['CEM'] = change_df['CEM'] * 100
    percent_change_df['PF'] = change_df['PF'] * 100
    percent_change_df['DLAC-p'] = change_df['DLAC-p'] * 100
    percent_change_df['DLAC-i'] = change_df['DLAC-i'] * 100
    percent_change_df['SLAC'] = change_df['SLAC'] * 100

    # remove decimal point
    percent_change_df[['CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']] = \
                        percent_change_df[['CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']].round(2)
    percent_change_df[['CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']] = \
        percent_change_df[['CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']].astype(str) + '%'

    return percent_change_df

In [65]:
std_results_df = pd.DataFrame.from_dict(std_dict, orient='index')
# display
std_results_df

,CEM,PF,DLAC-p,DLAC-i,SLAC
Thermal_Base,171.081433,178.515436,178.056245,208.053608,172.055218
2_Hr_BESS,273.513444,274.937517,270.567568,248.361973,244.930736
2_Hr_BESS_Fuelx2,251.761883,332.213551,228.786565,236.220512,229.136728
4_Hr_BESS,270.660793,293.876675,290.998994,266.102180,259.611979
4_Hr_BESS_Fuelx2,236.978937,337.169085,290.679411,234.800191,231.276138
4_Hr_BESS_Fuelx3,271.068296,275.619325,257.543657,232.095007,199.371365
4_Hr_BESS_Fuelx4,285.738862,289.637729,286.588826,267.841388,236.590699
6_Hr_BESS,252.561877,299.879369,279.552460,257.843289,259.251436
6_Hr_BESS_Fuelx2,261.770539,272.399097,239.898753,229.465028,224.937401
8_Hr_BESS,246.880117,333.681682,244.772631,273.660832,263.883467


In [76]:
std_results_change_df = percent_change_df(std_results_df)
# print df to latex .tex file
std_results_change_df.to_latex(latex_path + 'std_results_change.tex', 
    index=True, float_format="%.2f", escape=False, column_format='lccccccccc')
# display
std_results_change_df

,CEM,PF,DLAC-p,DLAC-i,SLAC
Thermal_Base,-4.16%,0.0%,-0.26%,16.55%,-3.62%
2_Hr_BESS,-0.52%,0.0%,-1.59%,-9.67%,-10.91%
2_Hr_BESS_Fuelx2,-24.22%,0.0%,-31.13%,-28.89%,-31.03%
4_Hr_BESS,-7.9%,0.0%,-0.98%,-9.45%,-11.66%
4_Hr_BESS_Fuelx2,-29.72%,0.0%,-13.79%,-30.36%,-31.41%
4_Hr_BESS_Fuelx3,-1.65%,0.0%,-6.56%,-15.79%,-27.66%
4_Hr_BESS_Fuelx4,-1.35%,0.0%,-1.05%,-7.53%,-18.31%
6_Hr_BESS,-15.78%,0.0%,-6.78%,-14.02%,-13.55%
6_Hr_BESS_Fuelx2,-3.9%,0.0%,-11.93%,-15.76%,-17.42%
8_Hr_BESS,-26.01%,0.0%,-26.64%,-17.99%,-20.92%


In [70]:

# create table of skew results with case names as index and models as columns
# Convert the skew_dict to a DataFrame
skew_results_df = pd.DataFrame.from_dict(skew_dict, orient='index').reset_index()
skew_results_df.columns = ['Case Name', 'CEM', 'PF', 'DLAC-p', 'DLAC-i', 'SLAC']

# Display the DataFrame
skew_results_df

,Case Name,CEM,PF,DLAC-p,DLAC-i,SLAC
0,Thermal_Base,26.066425,25.199467,25.416213,22.141384,25.584187
1,2_Hr_BESS,16.931639,16.915574,17.102711,18.640132,18.705657
2,2_Hr_BESS_Fuelx2,16.090825,13.939071,19.989363,19.207740,18.722886
3,4_Hr_BESS,16.702564,16.430859,16.649473,18.098828,18.112604
4,4_Hr_BESS_Fuelx2,13.620444,13.880762,15.604379,19.453284,18.621901
5,4_Hr_BESS_Fuelx3,16.000618,15.710992,16.521844,18.350755,19.428094
6,4_Hr_BESS_Fuelx4,15.567894,15.046759,15.221658,16.219607,16.013842
7,6_Hr_BESS,16.865545,15.787706,16.744650,18.627674,18.323459
8,6_Hr_BESS_Fuelx2,16.385872,16.347215,17.744747,19.721008,19.130260
9,8_Hr_BESS,16.801498,14.127274,17.819046,17.734437,17.945932


In [ ]:
skew_results_change_df = percent_change_df(skew_results_df)
# print df to latex .tex file
skew_results_change_df.to_latex(latex_path + 'skew_results_change.tex', 
    index=True, float_format="%.2f", escape=False, column_format='lccccccccc')
skew_results_change_df

,Case Name,CEM,PF,DLAC-p,DLAC-i,SLAC
0,Thermal_Base,3.44%,0.0%,0.86%,-12.14%,1.53%
1,2_Hr_BESS,0.09%,0.0%,1.11%,10.2%,10.58%
2,2_Hr_BESS_Fuelx2,15.44%,0.0%,43.41%,37.8%,34.32%
3,4_Hr_BESS,1.65%,0.0%,1.33%,10.15%,10.24%
4,4_Hr_BESS_Fuelx2,-1.88%,0.0%,12.42%,40.15%,34.16%
5,4_Hr_BESS_Fuelx3,1.84%,0.0%,5.16%,16.8%,23.66%
6,4_Hr_BESS_Fuelx4,3.46%,0.0%,1.16%,7.79%,6.43%
7,6_Hr_BESS,6.83%,0.0%,6.06%,17.99%,16.06%
8,6_Hr_BESS_Fuelx2,0.24%,0.0%,8.55%,20.64%,17.02%
9,8_Hr_BESS,18.93%,0.0%,26.13%,25.53%,27.03%
